# Invoking sagemaker endpoint
https://github.com/awslabs/amazon-sagemaker-examples/issues/589
https://towardsdatascience.com/sharing-your-sagemaker-model-eaa6c5d9ecb5
https://gdcoder.com/invoking-aws-sagemaker-model-endpoints-from-client-outside-of-aws/

Train xgboost with Python SDK

In [39]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
import io
import os
import datetime as dt
import pickle as pkl
import boto3
import json

import sagemaker
from sagemaker import get_execution_role
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner, IntegerParameter
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.predictor import csv_serializer, json_deserializer

##  use a third party account that is configured locally via aws cli with the profile name tester


In [99]:
session= boto3.Session(profile_name='tester', region_name='us-west-2')

In [100]:
client = session.client('sts') 

In [101]:
response = client.assume_role(
    RoleArn="arn:aws:iam::639464939556:role/validator",
    RoleSessionName="testSessionName",
    ExternalId='639464939556')
    # roles cannot be consumed by root account, must use user generated aws config!


In [104]:
# Create boto client and required to specify region_name
client = boto3.client('sagemaker-runtime',   
                      aws_access_key_id=response["Credentials"]["AccessKeyId"],    
                      aws_secret_access_key=response["Credentials"]["SecretAccessKey"],   
                      aws_session_token=response["Credentials"]["SessionToken"],
                      region_name='us-west-2')

In [47]:
payload = pd.read_csv('validation.csv')[:20]
payload.drop(payload.columns[0], axis=1, inplace=True)
payload_file = io.StringIO()
#saves dataframe to string IO
#This module implements a file-like class, StringIO, that reads and writes a string buffer (also known as memory files).

#io.StringIO is a class. It handles Unicode. It reflects the preferred Python 3 library structure.
payload.to_csv(payload_file, header = None, index = None)

In [49]:
payload_file.getvalue()

'0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,15,37.7504396332625,-122.444700219493\n0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,18,37.756673780548894,-122.464965318296\n0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,23,37.7822406949796,-122.481139437535\n0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,21,37.7106219642358,-122.448857670369\n0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,12,37.7354310008249,-122.443949690206\n0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,10,37.7674625261825,-122.41983289014401\n0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,13,37.8062216124785,-122.40833738850901\n0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,10,37.78373004094,-122.413753670487\n0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,15,37.7355926105862,-122.390416955474\n0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,7,37.7829514828801,-122.413259498719\n0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,16,37.7922168233794,-122.42945011546699\n1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,14,37.7254258506536,-122.434155711353

In [97]:
endpoint_name='sagemaker-xgboost-200204-1841-002-fc901d44'
response = client.invoke_endpoint(
    EndpointName= endpoint_name,
    Body= payload_file.getvalue(),
    ContentType = 'text/csv')

str_result = response['Body'].read().decode()
str_result


'16.0,16.0,16.0,36.0,21.0,21.0,16.0,1.0,21.0,7.0,16.0,21.0,7.0,21.0,16.0,7.0,20.0,21.0,16.0,16.0'

In [95]:
int_result=list(map(float, str_result.split(',')))
results = [int(i) for i in int_result]
results

[16, 16, 16, 36, 21, 21, 16, 1, 21, 7, 16, 21, 7, 21, 16, 7, 20, 21, 16, 16]